Данный код обрабатывает результаты тестирования алгоритмов из `cpp-effective-solution/`, собирает метрики и выводит их на экран.

In [11]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Image

In [49]:
def get_table(PRIM_time, PRIM_res,
              COST_time, COST_res, COST_cost,
              TYPES_time, TYPES_res, TYPES_cost,
              PARALL20_time, PARALL20_res, PARALL20_cost,
              PARALL100_time, PARALL100_res, PARALL100_cost,
              PARALL500_time, PARALL500_res, PARALL500_cost):
    """
    Данная функция получает значения всех метрик для данного тестирования (на конкретной карте, с конкретными control_set и types).
    
    Для этого необходимо в функцию передать массвы _res, _cost, _time для каждого алгоритма, участвующего в тестировании:
    PRIM (базовое решение, где стоимость примитива = длина), COST (базовое решение, стоимость примитива = стоимость коллизионного 
    следа), TYPES (альтернативное решение со склеиванием), PARALL20, 100, 500 (три варианта PARALL_T для разных T).
    (Для PRIM массив _cost не нужен, так как его стоимость несопоставима со стоимостью остальных алгоритмов: он ищет
    траектории кратчайшую, а не с наименьшим коллизионным следом)

    Эти массивы для каждого теста номер i содержат следюущее:
        _res[i] = 1, если путь в данном тесте алгоритм нашёл и 0 иначе,
        _time[i] = время работы (вне зависимости, нашелся ли путь) алгоритма на данном тесте,
        _cost[i] = стоимость решения, которое найдено алгоритмом.

    Будет возвращено в качестве результата таблица dataframe со всеми метриками.
    """

    assert (np.all(PRIM_res == COST_res) == 1)  # проверяем, что решенные тесты у PRIM и COST одинаковые (так должно быть, ведь
                                                # это базовые решения - отличается стоимость и все)#
    
    arr = []  # список строк таблицы

    # перебираем: название алгоритма, его массивы time, res, cost
    for name, time, res, cost in zip(["PRIM", "COST", "TYPES", "PARALL_20", "PARALL_100", "PARALL_500"],
                                     [PRIM_time, COST_time, TYPES_time, PARALL20_time, PARALL100_time, PARALL500_time], 
                                     [PRIM_res, COST_res, TYPES_res, PARALL20_res, PARALL100_res, PARALL500_res],
                                     [None, COST_cost, TYPES_cost, PARALL20_cost, PARALL100_cost, PARALL500_cost]):

        # Так как PRIM - базовое решение, то не может быть тестов, которые PRIM не решил,
        # а какой-то другой алгоритм решил...
        assert (res[PRIM_res == 0].sum() == 0)  # для этого все значения res, где PRIM_res = 0, должны быть нулями 
          
        line = []  # очередная строка таблицы
        line.append(name)  # имя алгоритма
        line.append(res.sum())  # количество решенных алгоритмом тестов
        line.append(round(res.sum() / PRIM_res.sum() * 100, 2))  # success rate (= доля от количетва тестов, решенных базовым решением), переводим в % и округляем до 2 знаков после запятой

         # на РЕШАЕМЫХ ТЕСТАХ (то есть решенных базовым решеним PRIM) вычисляем:
        line.append(f"{(time[PRIM_res==1].mean() * 1000):.2f}")  # среднее время работы алгоритма
        line.append(f"{(np.median(time[PRIM_res==1]) * 1000):.2f}")  # медианное время работы (оба времени в миллисекундах, мс)
        
        mask = (time > 0.01) & (PRIM_res == 1)  # тесты (маска в виде набора индексов), на которых алгоритм работал > 10 миллисекунд и которые были решаемы (базовым решеним PRIM) 
        speed_up = round((COST_time[mask] / time[mask]).mean(), 2)  # считаем ускорение относительно алгоритма COST на таких тестах
        line.append(speed_up)

        if cost is None:
            line.append(None)
        else:  # если текущий алгоритм не PRIM, то можно посчитать изменение стоимости
            cost_mask = (COST_cost > 0) & (res == 1)  # рассматриваем тесты, где стоимость пути у COST > 0, и на которых алгоритм нашёл путь
            line.append(round(((cost[cost_mask] / COST_cost[cost_mask]).mean() - 1) * 100, 2))  # считаем, на сколько в среднем дороже (в %) найденный алгоритмом путь, чем найденный COST
        
        low_mask = (PRIM_res == 1) & (TYPES_res == 0) & (PRIM_time > 0.01)  # маска тестов, решенных PRIM, но не решенных TYPES (такие вероятно есть, так как TYPES не даёт полноту), а также на которые PRIM потратил > 10 мс (чтобы тривиальные тесты не учитывать)
        line.append(round((time[low_mask] / PRIM_time[low_mask]).mean(), 2))  # считаем среднее замедление алгоритма на такой тестах
        
        arr.append(line)  # добавляем очередную строку таблицы
   
    df = pd.DataFrame(arr, columns=["Алгоритм", "Решено тестов", "Success rate, %",
                                    "Среднее время, мс", "Медианное время, мс", "Ускорение PRIM в тестах > 10 мс, раз",
                                    "Увеличиение стоимости, %", "Замедление, раз"])  # формируем dataframe
    return df

In [50]:
def get_metrics_from_file(file):
    """
    Данная функция должна собрать все метрики из файла file, который создаётся в процессе тестирования алгоритов. Такие файлы
    лежат в папке cpp-effective-solution/res.
    """

    # массивы для всех алгоритмов
    PRIM_time = []
    PRIM_res = []
    
    COST_time = []
    COST_res = []
    COST_cost = []
    
    TYPES_time = []
    TYPES_res = []
    TYPES_cost = []

    PARALL20_time = []
    PARALL20_res = []
    PARALL20_cost = []
    
    PARALL100_time = []
    PARALL100_res = []
    PARALL100_cost = []
    
    PARALL500_time = []
    PARALL500_res = []
    PARALL500_cost = []
    
    with open(file, "r") as f:
        while 1:  
            line = f.readline()  # построчно читаем файл
            if line == "":
                break  # дошли до конца -> выходим из цикла
                
            if "result" in line:  # если найдена строка результатов
                line1 = line
                line2 = f.readline()

                if ("result PRIM:" in line1) and ("time PRIMS:" in line2):  # проверяем, какой алгоритм нашёл путь согласно строке результатов          
                    PRIM_res.append(int(line1.split()[2]))
                    PRIM_time.append(float(line2.split()[2]))
                    continue

                if ("result COST:" in line1) and ("time COST:" in line2):                
                    COST_res.append(int(line1.split()[2]))
                    COST_time.append(float(line2.split()[2]))
                    COST_cost.append(float(line1.split()[4]))
                    continue

                if ("result TYPES:" in line1) and ("time TYPES:" in line2):                
                    TYPES_res.append(int(line1.split()[2]))
                    TYPES_time.append(float(line2.split()[2]))
                    TYPES_cost.append(float(line1.split()[4]))
                    continue

                if ("result PARALL:" in line1) and ("time PARALL 20:" in line2):                
                    PARALL20_res.append(int(line1.split()[2]))
                    PARALL20_time.append(float(line2.split()[3]))
                    PARALL20_cost.append(float(line1.split()[4]))
                    continue
                    
                if ("result PARALL:" in line1) and ("time PARALL 100:" in line2):                
                    PARALL100_res.append(int(line1.split()[2]))
                    PARALL100_time.append(float(line2.split()[3]))
                    PARALL100_cost.append(float(line1.split()[4]))
                    continue

                if ("result PARALL:" in line1) and ("time PARALL 500:" in line2):                
                    PARALL500_res.append(int(line1.split()[2]))
                    PARALL500_time.append(float(line2.split()[3]))
                    PARALL500_cost.append(float(line1.split()[4]))
                    continue

                raise Exception(line1, line2)  # если ни олин алгоритм не подошёл, что-то странное -> исключение

    PRIM_time = np.array(PRIM_time)  # превращаем массивы в numpy
    PRIM_res = np.array(PRIM_res, dtype=np.int32)
    
    COST_time = np.array(COST_time)
    COST_res = np.array(COST_res, dtype=np.int32)
    COST_cost = np.array(COST_cost)
    
    TYPES_time = np.array(TYPES_time)
    TYPES_res = np.array(TYPES_res, dtype=np.int32)
    TYPES_cost = np.array(TYPES_cost)

    PARALL20_time = np.array(PARALL20_time)
    PARALL20_res = np.array(PARALL20_res, dtype=np.int32)
    PARALL20_cost = np.array(PARALL20_cost)
    
    PARALL100_time = np.array(PARALL100_time)
    PARALL100_res = np.array(PARALL100_res, dtype=np.int32)
    PARALL100_cost = np.array(PARALL100_cost)

    PARALL500_time = np.array(PARALL500_time)
    PARALL500_res = np.array(PARALL500_res, dtype=np.int32)
    PARALL500_cost = np.array(PARALL500_cost)

    df = get_table(PRIM_time, PRIM_res,
                  COST_time, COST_res, COST_cost,
                  TYPES_time, TYPES_res, TYPES_cost,
                  PARALL20_time, PARALL20_res, PARALL20_cost,
                  PARALL100_time, PARALL100_res, PARALL100_cost,
                  PARALL500_time, PARALL500_res, PARALL500_cost)
    return df  # собираем таблицу из возвращаем её

In [51]:
maps = ["AR0700SR", "Milan_1_256", "Moscow_0_512", "Labyrinth", "WheelofWar", "w_woundedcoast"]  # названия всех карт, где тестировались алгоритмы
control_sets = ["main_control_set", "big_control_set", "short_control_set"]  # наборы control set, для которых было тестирование

for task_map in maps:
    img_file = "../cpp-effective-solution/maps/" + task_map + ".png"  # файл с картой

    results = []
    for cs in control_sets:
        res_file = "../cpp-effective-solution/res/" + task_map + "_" + cs + ".txt"
        df = get_metrics_from_file(res_file)
        results.append(df)  # собираем список всех результатов для данной карты

    display(HTML(f"<h1>Результаты карты {task_map}:</h1>"))  
    
    # пишем html-код, который рисует картинку с картой, справа от неё рисует три штуки dataframe - для каждого control set:
    html_content = f'''
    <br>
    <div style="display: flex;">
        <div style="flex: 1;">
            <img src="{img_file}" style="width: 400px; height: auto;"> 
        </div>
        <div style="flex: 1; margin-left: 20px;">
            <h5>Результаты для main_control_set</h5>
            {results[0].to_html()}
            <br>
            <h5>Результаты для big_control_set</h5>
            {results[1].to_html()}
            <br>
            <h5>Результаты для short_control_set</h5>
            {results[2].to_html()}
        </div>
    </div>
    '''
    # замечание: <br> ставит пробел в строку, <h3> делает текст жирным (как заголовок) - число можно увеличиивать, уменьшая
    # размер заголовка (<h1>, <h4>, ...)

    display(HTML(html_content))  # рисуем html-код


,Алгоритм,Решено тестов,"Success rate, %","Среднее время, мс","Медианное время, мс","Ускорение PRIM в тестах > 10 мс, раз","Увеличиение стоимости, %","Замедление, раз"
0,PRIM,2132,100.00,321.88,167.33,1.10,NaN,1.00
1,COST,2132,100.00,344.55,177.79,1.00,0.00,1.16
2,TYPES,1900,89.12,121.93,59.57,3.38,6.33,3.62
3,PARALL_20,2132,100.00,183.47,71.44,2.67,4.47,3.37
4,PARALL_100,2132,100.00,169.57,60.34,3.24,5.26,4.60
5,PARALL_500,2132,100.00,166.12,57.86,3.40,5.64,4.65
,Алгоритм,Решено тестов,"Success rate, %","Среднее время, мс","Медианное время, мс","Ускорение PRIM в тестах > 10 мс, раз","Увеличиение стоимости, %","Замедление, раз"
0,PRIM,2511,100.00,368.53,184.19,1.11,NaN,1.00
1,COST,2511,100.00,385.11,198.85,1.00,0.00,1.11
2,TYPES,1573,62.64,89.65,23.50,3.14,24.16,0.58


,Алгоритм,Решено тестов,"Success rate, %","Среднее время, мс","Медианное время, мс","Ускорение PRIM в тестах > 10 мс, раз","Увеличиение стоимости, %","Замедление, раз"
0,PRIM,1872,100.00,86.29,20.03,1.13,NaN,1.00
1,COST,1872,100.00,94.27,21.16,1.00,0.00,1.11
2,TYPES,1738,92.84,38.41,7.72,3.50,4.39,3.20
3,PARALL_20,1872,100.00,60.01,10.33,2.53,3.24,3.13
4,PARALL_100,1872,100.00,56.00,8.50,3.14,3.81,4.31
5,PARALL_500,1872,100.00,55.66,7.99,3.34,4.08,4.36
,Алгоритм,Решено тестов,"Success rate, %","Среднее время, мс","Медианное время, мс","Ускорение PRIM в тестах > 10 мс, раз","Увеличиение стоимости, %","Замедление, раз"
0,PRIM,2016,100.00,231.24,62.75,1.30,NaN,1.00
1,COST,2016,100.00,256.25,81.93,1.00,0.00,1.29
2,TYPES,1349,66.91,81.45,11.49,2.75,18.60,0.76


,Алгоритм,Решено тестов,"Success rate, %","Среднее время, мс","Медианное время, мс","Ускорение PRIM в тестах > 10 мс, раз","Увеличиение стоимости, %","Замедление, раз"
0,PRIM,4597,100.00,625.40,243.12,0.90,NaN,1.00
1,COST,4597,100.00,534.30,212.21,1.00,0.00,0.94
2,TYPES,4388,95.45,170.35,47.46,4.66,2.08,5.90
3,PARALL_20,4597,100.00,268.39,66.43,3.22,1.83,3.02
4,PARALL_100,4597,100.00,239.10,51.68,4.22,1.96,5.04
5,PARALL_500,4597,100.00,233.46,48.76,4.50,1.99,6.91
,Алгоритм,Решено тестов,"Success rate, %","Среднее время, мс","Медианное время, мс","Ускорение PRIM в тестах > 10 мс, раз","Увеличиение стоимости, %","Замедление, раз"
0,PRIM,4784,100.00,1991.87,635.77,1.11,NaN,1.00
1,COST,4784,100.00,1919.05,668.08,1.00,0.00,1.09
2,TYPES,3334,69.69,890.70,98.29,2.38,21.44,0.89


,Алгоритм,Решено тестов,"Success rate, %","Среднее время, мс","Медианное время, мс","Ускорение PRIM в тестах > 10 мс, раз","Увеличиение стоимости, %","Замедление, раз"
0,PRIM,8859,100.00,3346.53,2807.45,1.03,NaN,1.00
1,COST,8859,100.00,3528.65,2945.01,1.00,0.00,1.08
2,TYPES,8547,96.48,1123.37,793.31,3.81,1.51,17.56
3,PARALL_20,8859,100.00,1456.69,992.30,2.97,1.29,3.50
4,PARALL_100,8859,100.00,1267.12,803.11,3.72,1.37,6.33
5,PARALL_500,8859,100.00,1236.61,771.36,3.91,1.45,11.25
,Алгоритм,Решено тестов,"Success rate, %","Среднее время, мс","Медианное время, мс","Ускорение PRIM в тестах > 10 мс, раз","Увеличиение стоимости, %","Замедление, раз"
0,PRIM,9099,100.00,4582.18,4188.61,0.99,NaN,1.00
1,COST,9099,100.00,4570.60,4040.10,1.00,0.00,1.00
2,TYPES,6412,70.47,823.75,356.31,4.67,16.62,1.51


,Алгоритм,Решено тестов,"Success rate, %","Среднее время, мс","Медианное время, мс","Ускорение PRIM в тестах > 10 мс, раз","Увеличиение стоимости, %","Замедление, раз"
0,PRIM,8039,100.0,2700.69,1364.90,0.92,NaN,1.00
1,COST,8039,100.0,2568.42,1193.14,1.00,0.00,0.97
2,TYPES,7790,96.9,568.40,243.88,4.92,1.33,7.94
3,PARALL_20,8039,100.0,911.76,345.99,3.38,1.15,2.68
4,PARALL_100,8039,100.0,734.25,266.08,4.46,1.29,4.93
5,PARALL_500,8039,100.0,699.95,250.59,4.76,1.29,8.13
,Алгоритм,Решено тестов,"Success rate, %","Среднее время, мс","Медианное время, мс","Ускорение PRIM в тестах > 10 мс, раз","Увеличиение стоимости, %","Замедление, раз"
0,PRIM,8229,100.00,9256.05,5071.80,1.03,NaN,1.00
1,COST,8229,100.00,9297.11,4789.19,1.00,0.00,1.03
2,TYPES,5866,71.28,1857.22,628.55,3.54,18.14,1.26


,Алгоритм,Решено тестов,"Success rate, %","Среднее время, мс","Медианное время, мс","Ускорение PRIM в тестах > 10 мс, раз","Увеличиение стоимости, %","Замедление, раз"
0,PRIM,3416,100.00,212.88,171.10,1.08,NaN,1.00
1,COST,3416,100.00,229.96,182.61,1.00,0.00,1.11
2,TYPES,2794,81.79,76.24,53.69,3.58,15.78,1.75
3,PARALL_20,3416,100.00,138.92,66.95,2.81,10.22,2.35
4,PARALL_100,3416,100.00,132.18,57.24,3.32,11.91,2.77
5,PARALL_500,3416,100.00,130.62,55.26,3.46,12.90,2.76
,Алгоритм,Решено тестов,"Success rate, %","Среднее время, мс","Медианное время, мс","Ускорение PRIM в тестах > 10 мс, раз","Увеличиение стоимости, %","Замедление, раз"
0,PRIM,4061,100.00,466.07,386.33,1.10,NaN,1.00
1,COST,4061,100.00,504.17,412.91,1.00,0.00,1.10
2,TYPES,2459,60.55,111.41,55.27,3.63,25.60,0.78
